In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [2]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
def qtd_eventos(integer):
    return 45 - integer

qtd_eventos = list(map(qtd_eventos, df.isnull().sum(axis=1).tolist()))
df['qtd_eventos'] = qtd_eventos
df = df[df.qtd_eventos > 20]
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result,qtd_eventos
0,36864,ESPORTSTMNT02_2556988,19,1,2,1,3,24,2,23,...,0.71,4.2,1,0.00,2.5,1,1.00,7.0,0,28
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,0.80,4.8,1,0.00,0.8,5,0.60,3.2,1,31
3,36867,ESPORTSTMNT02_2557426,20,0,21,20,21,5,23,1,...,0.67,3.9,34,0.74,7.3,2,0.00,2.0,1,25
4,36868,ESPORTSTMNT02_2557463,19,1,2,1,2,4,20,24,...,0.40,2.4,2,1.00,24.0,1,1.00,6.0,1,23
6,36888,ESPORTSTMNT02_2557008,1,21,0,3,20,5,8,2,...,0.00,2.5,2,0.50,5.7,0,0.00,0.0,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4002,43354,ESPORTSTMNT01_3083705,19,21,1,20,22,5,21,4,...,0.00,2.3,2,0.50,5.3,11,0.36,3.2,0,24
4003,43355,ESPORTSTMNT01_3083711,19,21,24,20,21,23,26,22,...,0.30,3.0,3,0.67,8.0,1,1.00,15.0,1,23
4004,43356,ESPORTSTMNT01_3083725,19,21,3,20,21,5,23,1,...,0.55,2.4,5,0.60,4.2,6,0.17,2.9,0,23
4005,43357,ESPORTSTMNT01_3094483,0,2,4,1,2,3,24,6,...,0.67,4.2,24,0.54,3.7,4,0.00,1.3,0,23


In [3]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X

,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,qtd_eventos
0,19,1,2,1,3,24,2,23,6,1,...,14,0.71,4.2,1,0.00,2.5,1,1.00,7.0,28
2,19,2,20,24,2,4,20,3,6,1,...,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2,31
3,20,0,21,20,21,5,23,1,20,26,...,15,0.67,3.9,34,0.74,7.3,2,0.00,2.0,25
4,19,1,2,1,2,4,20,24,5,22,...,5,0.40,2.4,2,1.00,24.0,1,1.00,6.0,23
6,1,21,0,3,20,5,8,2,24,4,...,1,0.00,2.5,2,0.50,5.7,0,0.00,0.0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4002,19,21,1,20,22,5,21,4,3,1,...,3,0.00,2.3,2,0.50,5.3,11,0.36,3.2,24
4003,19,21,24,20,21,23,26,22,20,3,...,20,0.30,3.0,3,0.67,8.0,1,1.00,15.0,23
4004,19,21,3,20,21,5,23,1,15,4,...,11,0.55,2.4,5,0.60,4.2,6,0.17,2.9,23
4005,0,2,4,1,2,3,24,6,5,1,...,6,0.67,4.2,24,0.54,3.7,4,0.00,1.3,23


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained


c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [6]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [7]:
scores

,Model,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,Logistic Regression,0.899909,0.888889,0.912913,0.900741,0.899909
1,Support Vector Machine (Linear Kernel),0.898381,0.893175,0.903904,0.898507,0.898381
2,Support Vector Machine (RBF Kernel),0.901451,0.882521,0.924925,0.903226,0.901451
3,Decission Tree,0.865535,0.854227,0.879880,0.866864,0.865535
4,Adaboost,0.892361,0.894260,0.888889,0.891566,0.892361
5,Random Forest,0.901424,0.886957,0.918919,0.902655,0.901424
6,Gradient Boosting Classifier,0.904400,0.892128,0.918919,0.905325,0.904400
